In [3]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

import mplusdb
import pandas as pd
import time

In [5]:
mdb = mplusdb.MplusDatabase('../.db_config')

#### Get all key run counts, sliced by key level, and by spec

In [13]:
t0 = time.time()
#raise TimeoutError('This will launch a 15 min query. Is this what you are trying to do? ') #safety break
query = """SELECT spec, level, count(level) FROM 
    (SELECT run_id, spec, level FROM new_table INNER JOIN
    roster ON roster.run_id = new_table.id) as J 
    GROUP BY spec, level;
    """
print(query)
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute('use keyruns;')
cursor.execute(query)
ret = cursor.fetchall()
cursor.close()
conn.close()
print(time.time() - t0)

SELECT spec, level, count(level) FROM 
    (SELECT run_id, spec, level FROM new_table INNER JOIN
    roster ON roster.run_id = new_table.id) as J 
    GROUP BY spec, level;
    
420.32775616645813


In [6]:
#raise TimeoutError('This will overwrite things. Is this what you are trying to do? ') #safety break
df = pd.DataFrame(ret, columns = ['spec', 'key_level', 'num_keys'])
df.to_pickle('keynums_groupby_level_spec2.pkl')

#### Do spec counts for top 500 for each dungeon-week pair

In [6]:
# this view is already in the database. But putting it here just in case.
query_view = """
CREATE VIEW period_rank AS
SELECT * from (
SELECT id, dungeon, period, score, DENSE_RANK() OVER(PARTITION BY period, dungeon ORDER BY score DESC) as period_rank
from run) as subtable
WHERE period_rank <= 500
"""


count_query = """
SELECT period, dungeon, spec, count(spec) FROM period_rank
LEFT JOIN roster
ON period_rank.id = roster.run_id
GROUP BY period, dungeon, spec
"""

t0 = time.time()
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute('use keyruns;')
cursor.execute(count_query)
ret = cursor.fetchall()
cursor.close()
conn.close()
print(time.time() - t0)

33.04152989387512


In [7]:
print(len(ret))

13940


In [10]:
#raise TimeoutError('This will overwrite things. Is this what you are trying to do? ') #safety break
df = pd.DataFrame(ret, columns = ['period', 'dungeon', 'spec', 'num_keys'])
df.to_pickle('top500_keynums_groupby_period_dungeon_spec2.pkl')

In [2]:
def fn(run):
    run = 2
    print(run)
    
run = 3

fn(run)
print(run)

2
3
